In [1]:
import re
import numpy as np
import pandas as pd
from pyBioInfo.IO.File import FastaFile

# Merge circRNA expression matrix

In [3]:
def load_quantify(path):
    print("Loading %s" % path)
    d = pd.read_csv(path, sep="\t", header=None)
    d.columns = [
        "chrom", "start", "end", "name", "score", "strand", "thickStart", "thickEnd", 
        "itemRgb", "exonCount", "exonSizes", "exonOffsets", 
        "readNumber", "circType", "geneName", "isoformName", "index", "flankIntron", 
        "FPBcirc", "FPBlinear", "CIRCscore"]
    print("Loaded records: %d " % len(d))
    d = d[["CUFF" not in s for s in d["geneName"]]] # remove de novo circRNA
    d = d[[re.match("^chr([0-9]+|[XY])$", c) is not None for c in d["chrom"]]] # remove non-canonical chromosome circRNA
    print("After filtered: %d" % len(d))
    return d

samples = [
    "20210720_circRNAseq_oeHNRNPK_Rep1",
    "20210720_circRNAseq_oeHNRNPK_Rep2",
    "20210720_circRNAseq_oeNC_Rep1",
    "20210720_circRNAseq_oeNC_Rep2",
]
names = ["OE1", "OE2", "WT1", "WT2"]

array = []
for name, sample in zip(names, samples):
    print("-" * 80)
    print("Sample: %s" % sample)
    path = "results/2_CIRCexplorer/10_circ_quantify/%s.txt" % sample
    d = load_quantify(path)
    keys = [tuple(vs) for vs in d[["chrom", "start", "end", "strand", "exonCount", "exonSizes", "exonOffsets", "geneName", "isoformName", "flankIntron"]].values]
    assert len(keys) == len(set(keys))
    d2 = d[["readNumber", "FPBcirc", "FPBlinear", "CIRCscore"]]
    d2.index = keys
    d2.columns = ["%s.%s" % (c, name) for c in d2.columns]
    array.append(d2)

df = pd.concat(array, axis=1).fillna(0)
df.sort_index(inplace=True)
df = df[sorted(df.columns)]
anno = pd.DataFrame([list(row) for row in df.index], columns=["chrom", "start", "end", "strand", "exonCount", "exonSizes", "exonOffsets", "geneName", "isoformName", "flankIntron"])
df.index = np.arange(len(df))
df = df.merge(anno, left_index=True, right_index=True)
df.index = ["circRNA.%d" % i for i in range(len(df))]
df.index.name = "ID"
df["location"] = ["%s:%s-%s(%s)" % (chrom, start, end, strand) for chrom, start, end, strand in df[["chrom", 'start', 'end', 'strand']].values]
if True: # Added sequences
    lengths = []
    seqs = []
    with FastaFile("/home/zgchen/species/homo_sapiens/GRCh38.p13/GRCh38.canonical.genome.fa") as f:
        for chrom, start, end, sizes, offsets in df[["chrom", "start", "end", "exonSizes", "exonOffsets"]].values:
            sizes = list(map(int, sizes.split(",")))
            offsets = list(map(int, offsets.split(",")))
            vs = []
            for size, offset in zip(sizes, offsets):
                x = offset + start
                y = x + size
                vs.append(f.fast_fetch(chrom, x, y).upper())
            seq = "".join(vs)
            lengths.append(len(seq))
            seqs.append(seq)
    df["length"] = lengths
    df["sequence"] = seqs
df.to_csv("results/circ_rna_merged.tsv", sep="\t")

--------------------------------------------------------------------------------
Sample: 20210720_circRNAseq_oeHNRNPK_Rep1
Loading results/2_CIRCexplorer/10_circ_quantify/20210720_circRNAseq_oeHNRNPK_Rep1.txt
Loaded records: 29542 
After filtered: 26752
--------------------------------------------------------------------------------
Sample: 20210720_circRNAseq_oeHNRNPK_Rep2
Loading results/2_CIRCexplorer/10_circ_quantify/20210720_circRNAseq_oeHNRNPK_Rep2.txt
Loaded records: 34932 
After filtered: 30871
--------------------------------------------------------------------------------
Sample: 20210720_circRNAseq_oeNC_Rep1
Loading results/2_CIRCexplorer/10_circ_quantify/20210720_circRNAseq_oeNC_Rep1.txt
Loaded records: 16412 
After filtered: 15424
--------------------------------------------------------------------------------
Sample: 20210720_circRNAseq_oeNC_Rep2
Loading results/2_CIRCexplorer/10_circ_quantify/20210720_circRNAseq_oeNC_Rep2.txt
Loaded records: 20332 
After filtered: 18919


# Make circRNA bed file

In [4]:
df = pd.read_csv("results/circ_rna_merged.tsv", sep="\t", index_col=0)
with open("results/circ_rna_merged.bed", "w+") as fw:
    for name, (chrom, start, end, strand, exon_count, exon_sizes, exon_offsets) in zip(df.index, df[["chrom", "start", "end", "strand", "exonCount", "exonSizes", "exonOffsets"]].values):
        color = "0,0,255" if strand == "+" else "255,0,0"
        row = [chrom, start, end, name, ".", strand, start, start, color, exon_count, exon_sizes, exon_offsets]
        fw.write("\t".join(map(str, row)) + "\n")
! bgzip -f results/circ_rna_merged.bed
! tabix -p bed -f results/circ_rna_merged.bed.gz